In [2]:
import ee
import pathlib
import os
import rasterio
from tqdm import tqdm
import requests
import shutil

ModuleNotFoundError: No module named 'ee'

Register the Google Earth Engine in the `TreeAI` project

In [3]:
# Initialize the Earth Engine module and register this code in project 'TreeAI'
ee.Authenticate()
ee.Initialize(project='treeai-465719')

Run `scripts/upload_assets_to_gee.sh` to perform the upload.

In [4]:
def count_assets(parent_path):
    info = ee.data.listAssets({'parent': parent_path})
    assets = info.get('assets', [])
    count = 0
    for asset in assets:
        count += 1
        if asset['type'] == 'FOLDER':
            count += count_assets(asset['id'])
    return count


# Check if the upload is successful, if so, the number of shapefile assets should be the same as the number of UAV images.
gee_proj_root = 'users/bryce001006'
total = count_assets(gee_proj_root)
print(f"There are {total} assets under {gee_proj_root}")

There are 144 assets under users/bryce001006


In [44]:
def exportSatEmbed(asset_id, year, exp_scale=10, exp_folder='gee_export', maxPixels=1e13, ):
    img = ee.Image(asset_id)
    asset_name = os.path.basename(asset_id)

    # Get projection information of src UAV image
    ref_proj = img.projection()
    ref_crs = ref_proj.crs()
    aoi_rect = img.geometry().bounds(proj=ref_proj, maxError=1)

    # find corresponding satellite embedding dataset v1 image
    emb = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
           .filterDate(f'{year}-01-01', f'{year + 1}-01-01')
           .filterBounds(aoi_rect)
           .first()
           .clip(aoi_rect))

    # define the task parameters and submit the task to Google Cloud
    task = ee.batch.Export.image.toDrive(
        image=emb,
        description=f'Satellite embedding of {asset_name}',
        folder=exp_folder,
        fileNamePrefix=f'{asset_name}_SatEmbd_{year}',
        region=aoi_rect,
        crs=ref_crs,
        scale=exp_scale,
        maxPixels=maxPixels
    )
    task.start()
    # print(f'Submit the task of downloading satellite embedding data of {asset_name}_SatEmbd_{year}')

In [45]:
assets_list = ee.data.listAssets(gee_proj_root)['assets']

for year in range(2017, 2025):
    for asset in tqdm(assets_list, desc=f'Downloading Satellite Emnedding Dataset V1 in {year}'):
        exportSatEmbed(asset_id=asset['name'], year=year)

In [ ]:
def print_info(path):
    with rasterio.open(path) as src:
        print(f"\n==== {path} ====")
        print("CRS:", src.crs)
        print("Bounds:", src.bounds)
        print("Resolution:", src.res)
        print("Width x Height:", src.width, src.height)
        print("Transform:", src.transform)

In [4]:
treeai_swiss_folder = pathlib.Path('data/Swiss/TreeAI_Swiss')
treeai_swiss_folder.mkdir(parents=True, exist_ok=True)

uav_images_dir = pathlib.Path('data/Swiss/images')
uav_images_list = list(uav_images_dir.glob("*.tif"))

sat_embd_dir = pathlib.Path('data/Swiss/gee_export')
sat_embd_data = list(sat_embd_dir.glob("*.tif"))

NameError: name 'pathlib' is not defined